# Demo: Python–SQL Analysis
This notebook demonstrates how Python and SQL can be combined to query a simple, dimensionally modelled OLAP data warehouse using a star schema.

The dimension and fact tables within the warehouse were built using slightly modified raw CSV data from https://github.com/dbt-labs/jaffle-shop-data that has been pre-transformed, and loaded into 'warehouse.db' using Polars and Sqlite3.


In [24]:
import src.db_connection as dbc
import polars as pl

### Connecting to the data warehouse

In [25]:
connection_to_db = dbc.connect_to_db()

### Exploring the dimension and fact tables

In [26]:
pl.read_database(
    """
    SELECT name AS dw_table_name
    FROM sqlite_master WHERE type="table"
    """, connection_to_db)

dw_table_name
str
"""dim_date"""
"""dim_store"""
"""dim_customer"""
"""dim_product"""
"""fact_order"""
"""fact_order_item"""


#### Example: Date dimension table (dim_date)
The date dimension table extends a simple date value with additional columns that are useful to simplify queries, or target business-driven identifiers such as days, months or quarters. It utilises an integer-based "date_id" derived from the actual date as a primary key, which also acts as a foreign key for the "order_date_id" column in the order fact table.

In [27]:
pl.read_database(
    """
    SELECT * FROM dim_date
    LIMIT 5
    """, connection_to_db)

date_id,date_actual,day_num,day_name,month_num,month_name,year,qtr
i64,str,i64,str,i64,str,i64,i64
19010101,"""1901-01-01""",1,"""Tuesday""",1,"""January""",1901,1
19010102,"""1901-01-02""",2,"""Wednesday""",1,"""January""",1901,1
19010103,"""1901-01-03""",3,"""Thursday""",1,"""January""",1901,1
19010104,"""1901-01-04""",4,"""Friday""",1,"""January""",1901,1
19010105,"""1901-01-05""",5,"""Saturday""",1,"""January""",1901,1


#### Example: Store dimension table (dim_store)
This table provides information about physical stores. When the table was created as part of the ETL process, the "store_id" column was introduced as a surrogate primary key for the original store GUID. As with the other tables, the use of small integer-based surrogate keys can help to improve query and join performance.

In [28]:
pl.read_database(
    """
    SELECT * FROM dim_store
    LIMIT 5
    """, connection_to_db)

store_id,store_actual_id,name,tax_rate
i64,str,str,f64
1,"""a9128331-08d7-41a2-b615-21283e…","""Philadelphia""",0.06
2,"""a2a24e87-dec8-4f5d-9c9e-0e9849…","""Brooklyn""",0.04
3,"""e1a600e7-e70a-43a8-96fd-a6dd26…","""Chicago""",0.0625
4,"""0d16236e-9a7f-4cd6-a9ec-24b16f…","""San Francisco""",0.075
5,"""ca0b89cc-6d98-4b87-88e3-5a0d95…","""New Orleans""",0.04


#### Example: Order item fact table (fact_order_item)
This fact table acts a bridge between the product dimension and order fact tables. It utilises the surrogate "product_id" key from the product dimension table as a foreign key while retaining the original "order_id" present within the central order fact table.

As this fact table only acts as an intermediary and contains no numerical facts or measures, it is know as a "factless" fact table.

In [29]:
pl.read_database(
    """
    SELECT * FROM fact_order_item
    LIMIT 5
    """, connection_to_db)

order_item_id,order_item_actual_id,order_id,product_id
i64,str,str,i64
1,"""6136835c-cfa4-4308-bd65-440f7b…","""363ab5f2-78b4-4227-9333-9701ef…",6
2,"""1a5fc7f0-1573-454a-ae79-576673…","""363ab5f2-78b4-4227-9333-9701ef…",2
3,"""d32d93ca-250b-47b2-b640-cb482e…","""d6cdb882-aa8c-4b5a-a478-4321ef…",6
4,"""8dabb27c-fcc2-4513-8adf-886cdc…","""ca934a93-aca1-4aeb-9786-efb0da…",9
5,"""6849a62f-43ee-464c-8f1f-9d597f…","""ecd64383-c4df-41aa-b270-f3b110…",7


#### Example: Order fact table (fact_order)
This is the central fact table within the star schema of this data warehouse. Other than the "subtotal", "tax_paid" and "order_total" measures, the table holds an "order_id" as it's primary key and several foreign keys pointing to the dimension tables, such as "customer_id", "order_date_id" and "store_id".

In [30]:
pl.read_database(
    """
    SELECT * FROM fact_order
    LIMIT 5
    """, connection_to_db)

order_id,customer_id,order_date_id,store_id,subtotal,tax_paid,order_total
str,i64,i64,i64,i64,f64,f64
"""0002d5c6-b2d3-4da1-8ba0-18d877…",424,20220417,1,5,0.2,5.2
"""0003b2a1-8756-4bc7-99e2-a48347…",708,20250126,1,6,0.24,6.24
"""00068de4-42b4-47b2-ac56-9e6637…",284,20230331,1,4,0.24,4.24
"""00069750-235b-4cd8-a37b-ef6444…",491,20220401,1,4,0.16,4.16
"""0025ab1c-d156-4492-914e-7a2149…",182,20250124,1,7,0.42,7.42


### Querying the data warehouse
The queries below demonstrate how dimension and fact tables along with pre-aggregated data can simplify SQL statements, reducing the complexity of joins and the need for the extensive use of string and/or datetime functions.

#### What was the quarterly sales revenue for each store in 2024?

In [31]:
pl.read_database(
    """SELECT
    store.name AS store_name,
    SUM(CASE WHEN d.qtr = 1 THEN orders.order_total ELSE 0 END) AS Q1,
    SUM(CASE WHEN d.qtr = 2 THEN orders.order_total ELSE 0 END) AS Q2,
    SUM(CASE WHEN d.qtr = 3 THEN orders.order_total ELSE 0 END) AS Q3,
    SUM(CASE WHEN d.qtr = 4 THEN orders.order_total ELSE 0 END) AS Q4,
    SUM(orders.order_total) AS total_sales_2024
    FROM fact_order AS orders
    JOIN dim_store AS store ON store.store_id = orders.store_id
    JOIN dim_date AS d ON d.date_id = orders.order_date_id
    WHERE d.year = 2024
    GROUP BY store_name
    ORDER BY total_sales_2024 DESC
    """, connection_to_db)

store_name,Q1,Q2,Q3,Q4,total_sales_2024
str,f64,f64,f64,f64,f64
"""Brooklyn""",5268.11,5202.03,5200.86,5256.04,20927.04
"""San Francisco""",5047.57,4710.51,4890.49,5774.22,20422.79
"""Chicago""",5168.24,5556.35,4820.44,4667.49,20212.52
"""New Orleans""",5001.35,4553.84,5248.26,5315.5,20118.95
"""Los Angeles""",4511.42,4545.32,5228.53,5154.89,19440.16
"""Philadelphia""",4759.83,4686.48,4999.93,4969.75,19415.99


#### What was the highest selling product for each store for Q1 and Q2 in 2020?

In [32]:
pl.Config.set_tbl_rows(50)
pl.read_database(
    """
    -- get the total count of each product sold by store, and quarter.

    WITH sales AS (SELECT
    store.name AS store_name,
    d.qtr AS quarter,
    product.name AS product_name,
    product.type AS product_type,
    COUNT(*) AS units_sold
    FROM fact_order AS orders
    JOIN dim_date AS d ON d.date_id = orders.order_date_id
    JOIN dim_store AS store ON store.store_id = orders.store_id
    JOIN fact_order_item AS order_item ON order_item.order_id = orders.order_id
    JOIN dim_product AS product ON product.product_id = order_item.product_id
    WHERE
    d.year = 2020
    AND
    d.qtr IN (1,2)
    GROUP BY store_name, quarter, product_name, product_type
    ORDER by store_name, quarter),

    -- apply a row numbers based on the highest, to lowest sold products scoping it to each store, and quarter.

    ranked_units_sold AS (SELECT
    store_name,
    quarter,
    product_name,
    product_type,
    units_sold,
    RANK() OVER (PARTITION BY store_name, quarter ORDER BY units_sold DESC) AS sale_rank
    FROM sales)

    -- get the highest selling products for store, each quarter (i.e. sale_rank = 1) and pivot the result

    SELECT
    store_name,
    quarter,
    product_name AS highest_selling_product_name,
    units_sold
    FROM
    ranked_units_sold
    WHERE sale_rank = 1
    ORDER BY quarter
    """, connection_to_db)

store_name,quarter,highest_selling_product_name,units_sold
str,i64,str,i64
"""Brooklyn""",1,"""Vanilla ice""",127
"""Chicago""",1,"""Tangaroo""",125
"""Los Angeles""",1,"""For richer or pourover""",125
"""New Orleans""",1,"""Chai and mighty""",108
"""Philadelphia""",1,"""Chai and mighty""",121
"""San Francisco""",1,"""Chai and mighty""",116
"""Brooklyn""",2,"""Vanilla ice""",115
"""Chicago""",2,"""Adele-ade""",116
"""Los Angeles""",2,"""Tangaroo""",124


#### How did monthly sales change for the Los Angeles Store in 2023?

In [33]:
pl.read_database(
    """
    -- get monthly sales for the Los Angeles store

    WITH monthly_sales AS (
    SELECT
    d.month_num,
    d.month_name,
    SUM(orders.order_total) AS total_sales
    FROM fact_order AS orders
    JOIN dim_date AS d ON d.date_id = orders.order_date_id
    JOIN dim_store AS store ON store.store_id = orders.store_id
    WHERE
    store.name = "Los Angeles"
    AND
    d.year = 2023
    GROUP BY d.month_num, d.month_name
    ORDER BY d.month_num ASC)

    -- use LAG function to calculate previous months sales total, and compare month on month.

    SELECT
    month_name,
    total_sales,
    LAG(total_sales) OVER (ORDER BY month_num ASC) AS prev_month_sales,
    total_sales - LAG(total_sales) OVER (ORDER BY month_num ASC) AS monthly_sales_dif
    FROM monthly_sales
    ORDER BY month_num ASC
    """, connection_to_db)

month_name,total_sales,prev_month_sales,monthly_sales_dif
str,f64,f64,f64
"""January""",1570.59,null,null
"""February""",1545.07,1570.59,-25.52
"""March""",1903.44,1545.07,358.37
"""April""",1719.61,1903.44,-183.83
"""May""",1642.0,1719.61,-77.61
"""June""",1497.45,1642.0,-144.55
"""July""",1728.68,1497.45,231.23
"""August""",1551.84,1728.68,-176.84
"""September""",1651.43,1551.84,99.59


#### How many products were sold each month between 2020 and 2024?

In [34]:
pl.read_database(
    """
    SELECT
    d.month_name,
    d.year,
    COUNT(*) as units_sold
    FROM
    fact_order AS orders
    JOIN dim_date AS d ON d.date_id = orders.order_date_id
    WHERE d.year BETWEEN 2020 AND 2024
    GROUP BY d.year, d.month_num, d.month_name
    ORDER BY d.year, d.month_num ASC
    """, connection_to_db).pivot(
    index="month_name",
    on="year",
    values="units_sold"
)

month_name,2020,2021,2022,2023,2024
str,i64,i64,i64,i64,i64
"""January""",1023,982,967,949,925
"""February""",884,862,883,911,968
"""March""",994,960,953,941,942
"""April""",962,1015,960,948,926
"""May""",977,964,1007,948,980
"""June""",988,967,978,971,935
"""July""",1050,1020,932,978,1003
"""August""",969,962,1014,959,998
"""September""",954,950,980,922,949


#### Which customers made more than 5 purchases in the month of January this year?

In [35]:
pl.read_database(
     """
    SELECT
    customer.name AS customer_name,
    COUNT(*) as total_purchases_jan_2025
    FROM
    fact_order AS orders
    JOIN dim_date AS d ON d.date_id = orders.order_date_id
    JOIN dim_customer AS customer ON customer.customer_id = orders.customer_id
    WHERE
    d.year = 2025
    AND
    d.month_name = "January"
    GROUP BY customer_name
    HAVING COUNT(*) > 5
    ORDER BY total_purchases_jan_2025 DESC
    """, connection_to_db)

customer_name,total_purchases_jan_2025
str,i64
"""Margaret Lopez""",7
"""Henry Zamora""",7
"""Michael Figueroa""",6
"""Kevin Graham""",6
"""Cynthia Owens""",6
"""Christy Acosta""",6
"""Blake Matthews""",6
